# UiPath Workflow Analyzer Agent

Bu notebook, UiPath iş akışlarını (.xaml ve .json dosyalarını) detaylı olarak analiz eder:
- **İş Akışı Amacı**: Workflow'un ne yaptığını analiz eder
- **Hataları Tespit Et**: Hatalı geliştirmeleri ve eksiklikleri bulur
- **İyileştirme Önerileri**: Performans ve kod kalitesini artırmak için öneriler sunar

---

## Proje Yapısı
- **Main.xaml**: Ana workflow dosyası (BMI Calculator Automation)
- **project.json**: Proje konfigürasyonu ve dependency'ler

## 1. Gerekli Kütüphaneleri İçe Aktar

In [1]:
import xml.etree.ElementTree as ET
import json
import os
from pathlib import Path
from typing import Dict, List, Tuple, Any
from collections import defaultdict
import re
from datetime import datetime
from dataclasses import dataclass, asdict, field
import warnings
warnings.filterwarnings('ignore')

print("✅ Gerekli kütüphaneler başarıyla yüklendi")

✅ Gerekli kütüphaneler başarıyla yüklendi


## 2. Veri Modelleri (Data Classes) Tanımla

In [2]:
@dataclass
class Issue:
    """Tespit edilen sorun"""
    severity: str  # "Critical", "High", "Medium", "Low"
    category: str  # "Error Handling", "Performance", "Best Practice", etc.
    title: str
    description: str
    location: str  # Workflow'ta nerede
    solution: str  # Nasıl düzeltileceği

@dataclass
class Activity:
    """Workflow aktivitesi"""
    name: str
    type: str
    purpose: str
    inputs: List[str] = field(default_factory=list)
    outputs: List[str] = field(default_factory=list)
    is_error_handled: bool = False

@dataclass
class WorkflowAnalysis:
    """Workflow analiz sonuçları"""
    workflow_name: str
    workflow_purpose: str
    activities: List[Activity] = field(default_factory=list)
    variables: List[str] = field(default_factory=list)
    issues: List[Issue] = field(default_factory=list)
    recommendations: List[str] = field(default_factory=list)
    dependencies: Dict[str, str] = field(default_factory=dict)
    overall_health_score: float = 0.0

print("✅ Veri modelleri başarıyla tanımlandı")

✅ Veri modelleri başarıyla tanımlandı


## 3. XAML Dosyasını Parse Et

In [3]:
class XAMLParser:
    """XAML dosyasını parse eden sınıf"""
    
    # UiPath namespace'leri
    NAMESPACES = {
        'sap': 'http://schemas.microsoft.com/netfx/2009/xaml/activities/presentation',
        'sap2010': 'http://schemas.microsoft.com/netfx/2010/xaml/activities/presentation',
        'scg': 'clr-namespace:System.Collections.Generic;assembly=System.Private.CoreLib',
        'ui': 'http://schemas.uipath.com/workflow/activities',
        'uix': 'http://schemas.uipath.com/workflow/activities/uix',
        'ueab': 'clr-namespace:UiPath.Excel.Activities.Business;assembly=UiPath.Excel.Activities',
        'ue': 'clr-namespace:UiPath.Excel;assembly=UiPath.Excel.Activities',
        'x': 'http://schemas.microsoft.com/winfx/2006/xaml',
        'default': 'http://schemas.microsoft.com/netfx/2009/xaml/activities'
    }
    
    def __init__(self, xaml_path: str):
        self.xaml_path = xaml_path
        self.tree = None
        self.root = None
        self.namespaces = {}
        
    def parse(self) -> bool:
        """XAML dosyasını parse et"""
        try:
            self.tree = ET.parse(self.xaml_path)
            self.root = self.tree.getroot()
            return True
        except Exception as e:
            print(f"❌ XAML parse hatası: {e}")
            return False
    
    def get_activities(self) -> List[Dict[str, Any]]:
        """Tüm aktiviteleri çıkar"""
        activities = []
        
        # Tüm 'Activity' tag'larını bul
        for elem in self.root.iter():
            tag = elem.tag.split('}')[-1] if '}' in elem.tag else elem.tag
            
            # UiPath aktiviteleri
            if tag in ['Sequence', 'Flowchart', 'ForEachRow', 'If', 'While', 'Do', 
                       'Switch', 'Parallel', 'InvokeWorkflow']:
                activities.append({
                    'type': tag,
                    'name': elem.get('DisplayName', 'Unknown'),
                    'attributes': dict(elem.attrib)
                })
            
            # UI Automation aktiviteleri
            elif any(x in tag for x in ['NClick', 'NTypeInto', 'NGetText', 'NGetValue', 
                                         'NWaitElement', 'NAnchors']):
                activities.append({
                    'type': tag,
                    'name': elem.get('DisplayName', 'Unknown'),
                    'attributes': dict(elem.attrib)
                })
            
            # Excel aktiviteleri
            elif any(x in tag for x in ['ReadRange', 'WriteCell', 'ExcelApplicationCard', 
                                        'ExcelProcessScope', 'SaveExcelFile']):
                activities.append({
                    'type': tag,
                    'name': elem.get('DisplayName', 'Unknown'),
                    'attributes': dict(elem.attrib)
                })
        
        return activities
    
    def get_variables(self) -> List[Tuple[str, str]]:
        """Tüm değişkenleri çıkar"""
        variables = []
        
        for elem in self.root.iter():
            if 'Variable' in elem.tag:
                name = elem.get('Name', 'Unknown')
                var_type = elem.get('{http://schemas.microsoft.com/winfx/2006/xaml}TypeArguments', 'Unknown')
                variables.append((name, var_type))
        
        return variables
    
    def get_error_handlers(self) -> List[Dict[str, str]]:
        """Error handler'ları tespit et"""
        handlers = []
        
        for elem in self.root.iter():
            if any(x in elem.tag for x in ['TryCatch', 'Catch', 'Finally']):
                handlers.append({
                    'type': elem.tag.split('}')[-1],
                    'name': elem.get('DisplayName', 'Unknown')
                })
        
        return handlers
    
    def extract_text_content(self) -> str:
        """Metin içeriği çıkar (Log, Annotation vb)"""
        text_elements = []
        
        for elem in self.root.iter():
            if elem.text and elem.text.strip():
                text_elements.append(elem.text.strip())
        
        return '\n'.join(text_elements)

# Test et
xaml_file = "/Users/timocinapak/Documents/Code Repository/UiPath_Code_Analyser/xaml_files/Main.xaml"
parser = XAMLParser(xaml_file)

if parser.parse():
    print("✅ XAML dosyası başarıyla parse edildi")
    print(f"📊 Toplam aktivite sayısı: {len(parser.get_activities())}")
    print(f"📊 Değişken sayısı: {len(parser.get_variables())}")
    print(f"⚠️ Error handler sayısı: {len(parser.get_error_handlers())}")
else:
    print("❌ XAML parse hatası")

✅ XAML dosyası başarıyla parse edildi
📊 Toplam aktivite sayısı: 50
📊 Değişken sayısı: 3
⚠️ Error handler sayısı: 0


## 4. JSON Konfigürasyon Dosyasını Parse Et

In [4]:
class JSONConfigParser:
    """Project.json dosyasını parse eden sınıf"""
    
    def __init__(self, json_path: str):
        self.json_path = json_path
        self.config = None
    
    def parse(self) -> bool:
        """JSON dosyasını parse et"""
        try:
            with open(self.json_path, 'r', encoding='utf-8') as f:
                self.config = json.load(f)
            return True
        except Exception as e:
            print(f"❌ JSON parse hatası: {e}")
            return False
    
    def get_project_info(self) -> Dict[str, Any]:
        """Proje bilgilerini al"""
        if not self.config:
            return {}
        
        return {
            'name': self.config.get('name', 'Unknown'),
            'description': self.config.get('description', ''),
            'main': self.config.get('main', ''),
            'version': self.config.get('projectVersion', ''),
            'studio_version': self.config.get('studioVersion', ''),
        }
    
    def get_dependencies(self) -> Dict[str, str]:
        """Dependency'leri al"""
        if not self.config:
            return {}
        
        return self.config.get('dependencies', {})
    
    def get_runtime_options(self) -> Dict[str, Any]:
        """Runtime seçeneklerini al"""
        if not self.config:
            return {}
        
        return self.config.get('runtimeOptions', {})
    
    def get_design_options(self) -> Dict[str, Any]:
        """Design seçeneklerini al"""
        if not self.config:
            return {}
        
        return self.config.get('designOptions', {})

# Test et
json_file = "/Users/timocinapak/Documents/Code Repository/UiPath_Code_Analyser/xaml_files/project.json"
json_parser = JSONConfigParser(json_file)

if json_parser.parse():
    print("✅ JSON dosyası başarıyla parse edildi")
    project_info = json_parser.get_project_info()
    print(f"\n📋 Proje Bilgileri:")
    for key, value in project_info.items():
        print(f"   {key}: {value}")
    
    print(f"\n📦 Dependency'ler:")
    deps = json_parser.get_dependencies()
    for dep, version in deps.items():
        print(f"   {dep}: {version}")
else:
    print("❌ JSON parse hatası")

✅ JSON dosyası başarıyla parse edildi

📋 Proje Bilgileri:
   name: BMI Automation
   description: Blank Process
   main: Main.xaml
   version: 1.0.0
   studio_version: 24.10.1.0

📦 Dependency'ler:
   UiPath.Excel.Activities: [2.23.4]
   UiPath.Mail.Activities: [1.23.1]
   UiPath.System.Activities: [24.10.3]
   UiPath.Testing.Activities: [24.10.0]
   UiPath.UIAutomation.Activities: [24.10.0]


## 5. İş Akışı Analiz Motoru

In [5]:
class WorkflowAnalyzer:
    """Detaylı workflow analizi yapan sınıf"""
    
    def __init__(self, xaml_parser: XAMLParser, json_parser: JSONConfigParser):
        self.xaml_parser = xaml_parser
        self.json_parser = json_parser
        self.analysis = None
    
    def analyze(self) -> WorkflowAnalysis:
        """Detaylı analiz yap"""
        project_info = self.json_parser.get_project_info()
        
        self.analysis = WorkflowAnalysis(
            workflow_name=project_info.get('name', 'Unknown'),
            workflow_purpose=self._detect_workflow_purpose(),
            activities=self._extract_activities(),
            variables=self._extract_variables(),
            dependencies=self.json_parser.get_dependencies(),
        )
        
        # Hataları tespit et
        self.analysis.issues = self._detect_issues()
        
        # İyileştirme önerileri
        self.analysis.recommendations = self._generate_recommendations()
        
        # Sağlık skoru hesapla
        self.analysis.overall_health_score = self._calculate_health_score()
        
        return self.analysis
    
    def _detect_workflow_purpose(self) -> str:
        """İş akışının amacını algıla"""
        project_info = self.json_parser.get_project_info()
        description = project_info.get('description', '')
        
        # İlk bakış
        purpose = f"Proje: {project_info.get('name', 'Unknown')}\n"
        purpose += f"Açıklama: {description if description else 'Açıklama bulunmamaktadır.'}\n"
        
        # Aktivite tipine göre amacı tayin et
        activities = self.xaml_parser.get_activities()
        
        activity_types = set(act['type'] for act in activities)
        
        if any(x in str(activity_types) for x in ['NClick', 'NTypeInto', 'NGetText']):
            purpose += "- **Web Automation**: Browser'da UI otomasyonu yapılıyor\n"
        
        if any(x in str(activity_types) for x in ['ReadRange', 'WriteCell', 'ExcelApplicationCard']):
            purpose += "- **Excel İşleme**: Excel dosyalarıyla veri işleme yapılıyor\n"
        
        if 'ForEachRow' in str(activity_types):
            purpose += "- **Toplu İşleme**: Veri setinin her satırı üzerinde işlem yapılıyor\n"
        
        if 'If' in str(activity_types):
            purpose += "- **Koşullu İşleme**: Koşullara göre farklı işlemler uygulanıyor\n"
        
        return purpose
    
    def _extract_activities(self) -> List[Activity]:
        """Aktiviteleri çıkar"""
        activities = []
        xaml_activities = self.xaml_parser.get_activities()
        
        for act in xaml_activities:
            activity = Activity(
                name=act['name'],
                type=act['type'],
                purpose=self._get_activity_purpose(act['type']),
                is_error_handled=False
            )
            activities.append(activity)
        
        return activities
    
    def _get_activity_purpose(self, activity_type: str) -> str:
        """Aktivite türüne göre amacını açıkla"""
        purposes = {
            'Sequence': 'Adımları sırasıyla çalıştırır',
            'Flowchart': 'Flowchart mantığını kontrol eder',
            'ForEachRow': 'Veri tablosunun her satırında döngü yapar',
            'If': 'Koşullu branş oluşturur',
            'While': 'Koşul sağlanırken döngü yapar',
            'NClick': 'UI elementine tıklar',
            'NTypeInto': 'UI elementine metin yazı',
            'NGetText': 'UI elementinden metin okur',
            'ReadRange': 'Excel alanını okur',
            'WriteCell': 'Excel hücresine yazar',
            'ExcelApplicationCard': 'Excel dosyasını açar ve işler',
            'SaveExcelFile': 'Excel dosyasını kaydeder',
        }
        return purposes.get(activity_type, 'Bilinmeyen aktivite')
    
    def _extract_variables(self) -> List[str]:
        """Değişkenleri çıkar"""
        variables = self.xaml_parser.get_variables()
        return [f"{name} ({var_type})" for name, var_type in variables]
    
    def _detect_issues(self) -> List[Issue]:
        """Sorunları tespit et"""
        issues = []
        
        activities = self.xaml_parser.get_activities()
        variables = self.xaml_parser.get_variables()
        error_handlers = self.xaml_parser.get_error_handlers()
        
        # Sorun 1: Error Handler'ın olmaması
        if len(error_handlers) == 0 and len(activities) > 5:
            issues.append(Issue(
                severity='High',
                category='Error Handling',
                title='Error Handler Eksikliği',
                description='Workflow\'ta Try-Catch bloku bulunmamaktadır. Hata durumunda workflow başarısız olabilir.',
                location='Ana Sequence',
                solution='Try-Catch bloğu ekleyerek hata yönetimini iyileştirin.'
            ))
        
        # Sorun 2: Kullanılmayan değişkenler
        unused_vars = self._find_unused_variables()
        if unused_vars:
            for var in unused_vars:
                issues.append(Issue(
                    severity='Low',
                    category='Code Quality',
                    title='Kullanılmayan Değişken',
                    description=f'Değişken "{var}" tanımlanmış ancak kullanılmamış.',
                    location='Değişken Tanımı',
                    solution=f'Kullanılmayan "{var}" değişkenini silin veya kullanın.'
                ))
        
        # Sorun 3: Loop içinde UI işlemi
        if 'ForEachRow' in str([a['type'] for a in activities]):
            ui_in_loop = any(x in str([a['type'] for a in activities]) 
                           for x in ['NClick', 'NTypeInto', 'NGetText'])
            if ui_in_loop:
                issues.append(Issue(
                    severity='Medium',
                    category='Performance',
                    title='Loop içinde UI Otomasyonu',
                    description='ForEachRow döngüsü içinde UI otomasyonu yapılmaktadır. Bu, workflow\'ün çok yavaş çalışmasına neden olabilir.',
                    location='Calculate Sequence',
                    solution='Mümkünse UI işlemlerini optimize edin veya batch işleme kullanın.'
                ))
        
        # Sorun 4: DisplayName eksikliği
        unnamed_count = sum(1 for a in activities if a['name'] == 'Unknown')
        if unnamed_count > 0:
            issues.append(Issue(
                severity='Low',
                category='Best Practice',
                title='DisplayName Eksikliği',
                description=f'{unnamed_count} aktivitenin DisplayName özelliği ayarlanmamıştır.',
                location='Çeşitli Aktiviteler',
                solution='Tüm aktivitelere açıklayıcı DisplayName değerleri ekleyin.'
            ))
        
        return issues
    
    def _find_unused_variables(self) -> List[str]:
        """Kullanılmayan değişkenleri bul"""
        # Basit heuristic: Değişkenin DisplayName veya Text'te geçip geçmediğini kontrol et
        unused = []
        variables = self.xaml_parser.get_variables()
        
        xaml_content = ET.tostring(self.xaml_parser.root, encoding='unicode')
        
        for var_name, _ in variables:
            if var_name not in xaml_content or xaml_content.count(var_name) < 2:
                unused.append(var_name)
        
        return unused
    
    def _generate_recommendations(self) -> List[str]:
        """İyileştirme önerileri üret"""
        recommendations = []
        
        activities = self.xaml_parser.get_activities()
        dependencies = self.json_parser.get_dependencies()
        
        # Önerisi 1: Error Handling
        if not any(x in [a['type'] for a in activities] for x in ['TryCatch', 'Catch']):
            recommendations.append(
                "🔧 **Error Handling Ekleyin**: Try-Catch bloğu kullanarak runtime hatalarını yakalayın."
            )
        
        # Önerisi 2: Logging
        if not any('Log' in a['name'] for a in activities):
            recommendations.append(
                "📝 **Logging Ekleyin**: İş akışının çalışma durumunu izlemek için Log Message aktiviteleri ekleyin."
            )
        
        # Önerisi 3: Timeout
        browser_activities = [a for a in activities if 'N' in a['type'] and 'Application' in a['type']]
        if browser_activities and not any('timeout' in str(a).lower() for a in activities):
            recommendations.append(
                "⏱️ **Timeout Ayarlarını Kontrol Edin**: Browser işlemleri için uygun timeout değerleri belirleyin."
            )
        
        # Önerisi 4: Variable Scoping
        recommendations.append(
            "📌 **Variable Scoping**: Değişkenlerin sadece gerekli scope'ta tanımlanmasını sağlayın."
        )
        
        # Önerisi 5: Parameterization
        recommendations.append(
            "⚙️ **Parametreleştirme**: Hardcoded değerler yerine config dosyası kullanın (Config File Activity)."
        )
        
        # Önerisi 6: Monitoring
        recommendations.append(
            "📊 **Monitoring**: Business Process Analytics (BPA) ile workflow performansını izleyin."
        )
        
        return recommendations
    
    def _calculate_health_score(self) -> float:
        """Workflow sağlık skorunu hesapla (0-100)"""
        score = 100.0
        
        # Her sorun için puan azalt
        for issue in self.analysis.issues:
            if issue.severity == 'Critical':
                score -= 25
            elif issue.severity == 'High':
                score -= 15
            elif issue.severity == 'Medium':
                score -= 10
            elif issue.severity == 'Low':
                score -= 5
        
        return max(0, score)

# Test et
analyzer = WorkflowAnalyzer(parser, json_parser)
analysis = analyzer.analyze()

print("✅ Analiz tamamlandı!\n")
print(f"📊 Sağlık Skoru: {analysis.overall_health_score:.1f}/100")

✅ Analiz tamamlandı!

📊 Sağlık Skoru: 65.0/100


## 6. Rapor Üretici

In [6]:
class ReportGenerator:
    """Analiz raporunu oluşturan sınıf"""
    
    def __init__(self, analysis: WorkflowAnalysis):
        self.analysis = analysis
    
    def generate_markdown_report(self) -> str:
        """Markdown formatında rapor oluştur"""
        report = []
        
        report.append("# 📋 UiPath Workflow Analiz Raporu\n")
        report.append(f"**Oluşturma Tarihi**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        report.append("---\n")
        
        # 1. Özet
        report.append(self._generate_summary())
        
        # 2. İş Akışı Amacı
        report.append(self._generate_purpose())
        
        # 3. Aktiviteler
        report.append(self._generate_activities_section())
        
        # 4. Değişkenler
        report.append(self._generate_variables_section())
        
        # 5. Sorunlar
        report.append(self._generate_issues_section())
        
        # 6. İyileştirme Önerileri
        report.append(self._generate_recommendations_section())
        
        # 7. Bağımlılıklar
        report.append(self._generate_dependencies_section())
        
        return '\n'.join(report)
    
    def _generate_summary(self) -> str:
        """Özet bölümü"""
        health_color = self._get_health_color()
        
        summary = f"""
## 📊 Özet

| Metrik | Değer |
|--------|-------|
| **Workflow Adı** | {self.analysis.workflow_name} |
| **Sağlık Skoru** | {health_color} {self.analysis.overall_health_score:.1f}/100 |
| **Toplam Aktivite** | {len(self.analysis.activities)} |
| **Toplam Değişken** | {len(self.analysis.variables)} |
| **Tespit Edilen Sorun** | {len(self.analysis.issues)} |
| **İyileştirme Önerisi** | {len(self.analysis.recommendations)} |
"""
        return summary
    
    def _get_health_color(self) -> str:
        """Sağlık skoruna göre emoji"""
        score = self.analysis.overall_health_score
        if score >= 80:
            return "🟢"
        elif score >= 60:
            return "🟡"
        else:
            return "🔴"
    
    def _generate_purpose(self) -> str:
        """İş akışı amacı"""
        return f"""
## 🎯 İş Akışı Amacı

{self.analysis.workflow_purpose}
"""
    
    def _generate_activities_section(self) -> str:
        """Aktiviteler bölümü"""
        if not self.analysis.activities:
            return "\n## 📌 Aktiviteler\n\nHiçbir aktivite tespit edilmedi.\n"
        
        section = "\n## 📌 Aktiviteler\n\n"
        section += f"**Toplam: {len(self.analysis.activities)}**\n\n"
        
        # Aktiviteleri tipe göre grupla
        activities_by_type = defaultdict(list)
        for act in self.analysis.activities:
            activities_by_type[act.type].append(act)
        
        for act_type, acts in activities_by_type.items():
            section += f"### {act_type} ({len(acts)})\n"
            for act in acts:
                section += f"- **{act.name}**: {act.purpose}\n"
            section += "\n"
        
        return section
    
    def _generate_variables_section(self) -> str:
        """Değişkenler bölümü"""
        if not self.analysis.variables:
            return "\n## 🔤 Değişkenler\n\nHiçbir değişken tanımlanmamıştır.\n"
        
        section = "\n## 🔤 Değişkenler\n\n"
        section += f"**Toplam: {len(self.analysis.variables)}**\n\n"
        
        for var in self.analysis.variables:
            section += f"- `{var}`\n"
        
        return section
    
    def _generate_issues_section(self) -> str:
        """Sorunlar bölümü"""
        if not self.analysis.issues:
            return "\n## ⚠️ Sorunlar\n\n✅ Hiçbir sorun tespit edilmedi!\n"
        
        section = "\n## ⚠️ Tespit Edilen Sorunlar\n\n"
        section += f"**Toplam: {len(self.analysis.issues)}**\n\n"
        
        # Severity'ye göre grupla
        by_severity = defaultdict(list)
        for issue in self.analysis.issues:
            by_severity[issue.severity].append(issue)
        
        severity_order = ['Critical', 'High', 'Medium', 'Low']
        for severity in severity_order:
            if severity not in by_severity:
                continue
            
            severity_emoji = {
                'Critical': '🔴',
                'High': '🟠',
                'Medium': '🟡',
                'Low': '🟢'
            }[severity]
            
            section += f"### {severity_emoji} {severity} ({len(by_severity[severity])})\n\n"
            
            for issue in by_severity[severity]:
                section += f"**{issue.title}**\n"
                section += f"- **Kategori**: {issue.category}\n"
                section += f"- **Yer**: {issue.location}\n"
                section += f"- **Problem**: {issue.description}\n"
                section += f"- **Çözüm**: {issue.solution}\n\n"
        
        return section
    
    def _generate_recommendations_section(self) -> str:
        """İyileştirme önerileri"""
        if not self.analysis.recommendations:
            return "\n## 💡 İyileştirme Önerileri\n\nÖnerilen iyileştirme bulunmamaktadır.\n"
        
        section = "\n## 💡 İyileştirme Önerileri\n\n"
        
        for i, rec in enumerate(self.analysis.recommendations, 1):
            section += f"{i}. {rec}\n"
        
        return section
    
    def _generate_dependencies_section(self) -> str:
        """Bağımlılıklar"""
        if not self.analysis.dependencies:
            return "\n## 📦 Bağımlılıklar\n\nHiçbir bağımlılık tanımlanmamıştır.\n"
        
        section = "\n## 📦 Bağımlılıklar\n\n"
        
        for dep, version in self.analysis.dependencies.items():
            section += f"- `{dep}`: {version}\n"
        
        return section
    
    def save_report(self, output_path: str) -> bool:
        """Raporu dosyaya kaydet"""
        try:
            report = self.generate_markdown_report()
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(report)
            return True
        except Exception as e:
            print(f"❌ Rapor kaydedilirken hata: {e}")
            return False

# Test et
report_generator = ReportGenerator(analysis)
report_content = report_generator.generate_markdown_report()
print(report_content)

# 📋 UiPath Workflow Analiz Raporu

**Oluşturma Tarihi**: 2025-12-11 15:00:34

---


## 📊 Özet

| Metrik | Değer |
|--------|-------|
| **Workflow Adı** | BMI Automation |
| **Sağlık Skoru** | 🟡 65.0/100 |
| **Toplam Aktivite** | 50 |
| **Toplam Değişken** | 3 |
| **Tespit Edilen Sorun** | 4 |
| **İyileştirme Önerisi** | 5 |


## 🎯 İş Akışı Amacı

Proje: BMI Automation
Açıklama: Blank Process
- **Web Automation**: Browser'da UI otomasyonu yapılıyor
- **Excel İşleme**: Excel dosyalarıyla veri işleme yapılıyor
- **Toplu İşleme**: Veri setinin her satırı üzerinde işlem yapılıyor
- **Koşullu İşleme**: Koşullara göre farklı işlemler uygulanıyor



## 📌 Aktiviteler

**Toplam: 50**

### Sequence (14)
- **Main Sequence**: Adımları sırasıyla çalıştırır
- **Fetch Data**: Adımları sırasıyla çalıştırır
- **Do**: Adımları sırasıyla çalıştırır
- **Do**: Adımları sırasıyla çalıştırır
- **Calculate**: Adımları sırasıyla çalıştırır
- **Do**: Adımları sırasıyla çalıştırır
- **Body**: Adımları sırasıyla ç

## 7. Ana Analiz Agent'ı Çalıştır

In [7]:
def run_workflow_analysis(xaml_path: str, json_path: str, output_report_path: str = None) -> Tuple[WorkflowAnalysis, str]:
    """
    Ana analiz fonksiyonu - Workflow'ları analiz et ve rapor oluştur
    
    Args:
        xaml_path: XAML dosyasının yolu
        json_path: JSON dosyasının yolu
        output_report_path: Rapor dosyasının kaydedileceği yol (isteğe bağlı)
    
    Returns:
        Tuple[WorkflowAnalysis, str]: Analiz sonuçları ve rapor metni
    """
    
    print("🚀 UiPath Workflow Analyzer başlanıyor...\n")
    print("="*60)
    
    # 1. Parse işlemleri
    print("\n1️⃣ Dosyalar Parse Ediliyor...")
    
    xaml_parser = XAMLParser(xaml_path)
    if not xaml_parser.parse():
        return None, "XAML parse hatası"
    print("   ✅ XAML başarıyla parse edildi")
    
    json_parser = JSONConfigParser(json_path)
    if not json_parser.parse():
        return None, "JSON parse hatası"
    print("   ✅ JSON başarıyla parse edildi")
    
    # 2. Analiz
    print("\n2️⃣ Workflow Analiz Ediliyor...")
    analyzer = WorkflowAnalyzer(xaml_parser, json_parser)
    analysis = analyzer.analyze()
    print("   ✅ Analiz tamamlandı")
    
    # 3. Rapor Üretimi
    print("\n3️⃣ Rapor Oluşturuluyor...")
    report_generator = ReportGenerator(analysis)
    report_content = report_generator.generate_markdown_report()
    print("   ✅ Rapor oluşturuldu")
    
    # 4. Raporu Kaydet
    if output_report_path:
        print("\n4️⃣ Rapor Kaydediliyor...")
        if report_generator.save_report(output_report_path):
            print(f"   ✅ Rapor kaydedildi: {output_report_path}")
        else:
            print("   ❌ Rapor kaydedilemedi")
    
    print("\n" + "="*60)
    print("✅ Analiz başarıyla tamamlandı!\n")
    
    return analysis, report_content

# Analiz Çalıştır
xaml_file_path = "/Users/timocinapak/Documents/Code Repository/UiPath_Code_Analyser/xaml_files/Main.xaml"
json_file_path = "/Users/timocinapak/Documents/Code Repository/UiPath_Code_Analyser/xaml_files/project.json"
report_output_path = "/Users/timocinapak/Documents/Code Repository/UiPath_Code_Analyser/workflow_analysis_report.md"

analysis, report = run_workflow_analysis(xaml_file_path, json_file_path, report_output_path)

🚀 UiPath Workflow Analyzer başlanıyor...


1️⃣ Dosyalar Parse Ediliyor...
   ✅ XAML başarıyla parse edildi
   ✅ JSON başarıyla parse edildi

2️⃣ Workflow Analiz Ediliyor...
   ✅ Analiz tamamlandı

3️⃣ Rapor Oluşturuluyor...
   ✅ Rapor oluşturuldu

4️⃣ Rapor Kaydediliyor...
   ✅ Rapor kaydedildi: /Users/timocinapak/Documents/Code Repository/UiPath_Code_Analyser/workflow_analysis_report.md

✅ Analiz başarıyla tamamlandı!



## 8. Detaylı Analiz Sonuçları

In [8]:
if analysis:
    print("\n" + "="*60)
    print("📊 DETAYLI ANALIZ SONUÇLARI")
    print("="*60)
    
    # Özet İstatistikler
    print("\n📈 İstatistikler:")
    print(f"   • Workflow Adı: {analysis.workflow_name}")
    print(f"   • Toplam Aktivite: {len(analysis.activities)}")
    print(f"   • Toplam Değişken: {len(analysis.variables)}")
    print(f"   • Tespit Edilen Sorun: {len(analysis.issues)}")
    print(f"   • Sağlık Skoru: {analysis.overall_health_score:.1f}/100")
    
    # Aktiviteler
    print("\n📌 Aktivite Özeti:")
    from collections import Counter
    activity_types = Counter(act.type for act in analysis.activities)
    for act_type, count in activity_types.most_common():
        print(f"   • {act_type}: {count}")
    
    # Sorunları Göster
    if analysis.issues:
        print("\n⚠️ TESPIT EDILEN SORUNLAR:")
        
        # Severity'ye göre grupla
        by_severity = {}
        for issue in analysis.issues:
            if issue.severity not in by_severity:
                by_severity[issue.severity] = []
            by_severity[issue.severity].append(issue)
        
        for severity in ['Critical', 'High', 'Medium', 'Low']:
            if severity in by_severity:
                print(f"\n   🔴 {severity.upper()} ({len(by_severity[severity])}):")
                for issue in by_severity[severity]:
                    print(f"      • {issue.title}")
                    print(f"        {issue.description}")
    
    # İyileştirme Önerileri
    print("\n💡 İYİLEŞTİRME ÖNERİLERİ:")
    for i, rec in enumerate(analysis.recommendations, 1):
        print(f"   {i}. {rec}")
    
    print("\n" + "="*60)


📊 DETAYLI ANALIZ SONUÇLARI

📈 İstatistikler:
   • Workflow Adı: BMI Automation
   • Toplam Aktivite: 50
   • Toplam Değişken: 3
   • Tespit Edilen Sorun: 4
   • Sağlık Skoru: 65.0/100

📌 Aktivite Özeti:
   • Sequence: 14
   • NTypeInto: 3
   • NTypeInto.Target: 3
   • NTypeInto.VerifyOptions: 3
   • NClick: 3
   • NClick.Target: 3
   • NClick.VerifyOptions: 3
   • ExcelProcessScopeX: 2
   • ExcelProcessScopeX.Body: 2
   • ExcelApplicationCard: 2
   • ExcelApplicationCard.Body: 2
   • WriteCellX: 2
   • SaveExcelFileX: 2
   • Flowchart: 1
   • ReadRange: 1
   • ForEachRow: 1
   • If: 1
   • NGetText: 1
   • NGetText.Target: 1

⚠️ TESPIT EDILEN SORUNLAR:

   🔴 HIGH (1):
      • Error Handler Eksikliği
        Workflow'ta Try-Catch bloku bulunmamaktadır. Hata durumunda workflow başarısız olabilir.

   🔴 MEDIUM (1):
      • Loop içinde UI Otomasyonu
        ForEachRow döngüsü içinde UI otomasyonu yapılmaktadır. Bu, workflow'ün çok yavaş çalışmasına neden olabilir.

   🔴 LOW (2):
      • K

## 9. İnteraktif Analiz Paneli (Optional)

In [9]:
# İnteraktif fonksiyonlar
def show_issues_by_severity(analysis: WorkflowAnalysis, severity: str = None):
    """Belirli severity seviyesine göre sorunları göster"""
    if not analysis.issues:
        print("✅ Hiçbir sorun tespit edilmedi!")
        return
    
    if severity:
        issues = [i for i in analysis.issues if i.severity == severity]
        print(f"\n🔍 {severity} Seviyesi Sorunlar ({len(issues)}):\n")
    else:
        issues = analysis.issues
        print(f"\n🔍 Tüm Sorunlar ({len(issues)}):\n")
    
    for i, issue in enumerate(issues, 1):
        print(f"{i}. {issue.title}")
        print(f"   Kategori: {issue.category}")
        print(f"   Yer: {issue.location}")
        print(f"   Problem: {issue.description}")
        print(f"   ✅ Çözüm: {issue.solution}")
        print()

def get_issue_statistics(analysis: WorkflowAnalysis):
    """Sorun istatistiklerini göster"""
    if not analysis.issues:
        print("✅ Sorun istatistiği: Hiçbir sorun tespit edilmedi!")
        return
    
    print("\n📊 SORUN İSTATİSTİKLERİ\n")
    
    # Severity
    severity_counts = {}
    for issue in analysis.issues:
        severity_counts[issue.severity] = severity_counts.get(issue.severity, 0) + 1
    
    print("📌 Severity'ye göre:")
    for severity in ['Critical', 'High', 'Medium', 'Low']:
        count = severity_counts.get(severity, 0)
        if count > 0:
            emoji = '🔴' if severity == 'Critical' else '🟠' if severity == 'High' else '🟡' if severity == 'Medium' else '🟢'
            print(f"   {emoji} {severity}: {count}")
    
    # Category
    category_counts = {}
    for issue in analysis.issues:
        category_counts[issue.category] = category_counts.get(issue.category, 0) + 1
    
    print("\n📚 Kategoriye göre:")
    for category, count in sorted(category_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"   • {category}: {count}")

def get_activity_statistics(analysis: WorkflowAnalysis):
    """Aktivite istatistiklerini göster"""
    if not analysis.activities:
        print("❌ Aktivite bulunamadı")
        return
    
    print("\n📌 AKTİVİTE İSTATİSTİKLERİ\n")
    print(f"Toplam Aktivite: {len(analysis.activities)}\n")
    
    type_counts = {}
    for act in analysis.activities:
        type_counts[act.type] = type_counts.get(act.type, 0) + 1
    
    print("Aktivite Türü Dağılımı:")
    for act_type, count in sorted(type_counts.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / len(analysis.activities)) * 100
        print(f"   • {act_type}: {count} ({percentage:.1f}%)")

# Örnek kullanımlar
if analysis:
    print("\n" + "="*60)
    print("📊 İNTERAKTİF ANALİZ PANELI")
    print("="*60)
    
    # Sorun İstatistikleri
    get_issue_statistics(analysis)
    
    # Aktivite İstatistikleri
    get_activity_statistics(analysis)
    
    print("\n" + "="*60)


📊 İNTERAKTİF ANALİZ PANELI

📊 SORUN İSTATİSTİKLERİ

📌 Severity'ye göre:
   🟠 High: 1
   🟡 Medium: 1
   🟢 Low: 2

📚 Kategoriye göre:
   • Error Handling: 1
   • Code Quality: 1
   • Performance: 1
   • Best Practice: 1

📌 AKTİVİTE İSTATİSTİKLERİ

Toplam Aktivite: 50

Aktivite Türü Dağılımı:
   • Sequence: 14 (28.0%)
   • NTypeInto: 3 (6.0%)
   • NTypeInto.Target: 3 (6.0%)
   • NTypeInto.VerifyOptions: 3 (6.0%)
   • NClick: 3 (6.0%)
   • NClick.Target: 3 (6.0%)
   • NClick.VerifyOptions: 3 (6.0%)
   • ExcelProcessScopeX: 2 (4.0%)
   • ExcelProcessScopeX.Body: 2 (4.0%)
   • ExcelApplicationCard: 2 (4.0%)
   • ExcelApplicationCard.Body: 2 (4.0%)
   • WriteCellX: 2 (4.0%)
   • SaveExcelFileX: 2 (4.0%)
   • Flowchart: 1 (2.0%)
   • ReadRange: 1 (2.0%)
   • ForEachRow: 1 (2.0%)
   • If: 1 (2.0%)
   • NGetText: 1 (2.0%)
   • NGetText.Target: 1 (2.0%)

